# Import libraries

In [14]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split
import numpy as np

# Load and Preprocess data


In [16]:
# Set random seeds for reproducibility
np.random.seed(42)
torch.manual_seed(42)

# Step 1: Generate Synthetic Data (Uncorrelated Patterns for Outputs)
num_samples = 1000
input_features = 4
output_features = 4  # Since we have four continuous outputs

# Generate random input data
X = np.random.rand(num_samples, input_features)

# Generate unrelated random outputs
Y = np.random.rand(num_samples, output_features) * 100  # Outputs scaled up for diversity

# Convert data to PyTorch tensors
X_tensor = torch.tensor(X, dtype=torch.float32)
Y_tensor = torch.tensor(Y, dtype=torch.float32)

# Create a TensorDataset and DataLoader
dataset = TensorDataset(X_tensor, Y_tensor)
train_size = int(0.8 * num_samples)
test_size = num_samples - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)




# Define MTL model

In [17]:
# Step 2: Define the Multi-Task Regression Model
class MultiTaskRegressionModel(nn.Module):
    def __init__(self, input_dim, shared_dim, task_dim, num_tasks):
        super(MultiTaskRegressionModel, self).__init__()
        
        # Shared layers
        self.shared = nn.Sequential(
            nn.Linear(input_dim, shared_dim),
            nn.ReLU(),
            nn.Linear(shared_dim, shared_dim),
            nn.ReLU()
        )
        
        # Separate heads for each regression task
        self.heads = nn.ModuleList([nn.Linear(shared_dim, task_dim) for _ in range(num_tasks)])

    def forward(self, x):
        shared_output = self.shared(x)
        outputs = [head(shared_output) for head in self.heads]
        return torch.cat(outputs, dim=1)

# Instantiate the model
input_dim = input_features
shared_dim = 64
task_dim = 1
num_tasks = 4
model = MultiTaskRegressionModel(input_dim, shared_dim, task_dim, num_tasks)



# Define Loss function and Optimizer


In [18]:
# Step 3: Define Loss Function and Optimizer for Regression
criterion = nn.MSELoss()  # Mean Squared Error Loss for regression
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop

In [19]:
# Step 4: Training Loop
num_epochs = 20
for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    for X_batch, Y_batch in train_loader:
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, Y_batch)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * X_batch.size(0)
    
    train_loss /= train_size
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {train_loss:.4f}')



Epoch 1/20, Loss: 3189.6110
Epoch 2/20, Loss: 3034.0648
Epoch 3/20, Loss: 2653.4546
Epoch 4/20, Loss: 1974.2773
Epoch 5/20, Loss: 1232.6321
Epoch 6/20, Loss: 920.0150
Epoch 7/20, Loss: 886.6596
Epoch 8/20, Loss: 880.9904
Epoch 9/20, Loss: 877.3647
Epoch 10/20, Loss: 873.5608
Epoch 11/20, Loss: 870.2516
Epoch 12/20, Loss: 866.6233
Epoch 13/20, Loss: 863.6417
Epoch 14/20, Loss: 860.3048
Epoch 15/20, Loss: 857.2151
Epoch 16/20, Loss: 854.1353
Epoch 17/20, Loss: 851.4882
Epoch 18/20, Loss: 848.6302
Epoch 19/20, Loss: 846.0730
Epoch 20/20, Loss: 843.7204


# Evaluation on Test Data

In [22]:
# Step 5: Evaluation on Test Data
model.eval()
test_loss = 0.0
with torch.no_grad():
    for X_batch, Y_batch in test_loader:
        outputs = model(X_batch)
        loss = criterion(outputs, Y_batch)
        test_loss += loss.item() * X_batch.size(0)

test_loss /= test_size
print(f'Test Loss: {test_loss:.4f}')

Test Loss: 908.8706
